# Demo

Xarray-simlab is deeply integrated with other tools (Python, Jupyter + widgets, Xarray, Dask, Zarr, etc.) that together provide an interactive computing environment suited for the development of "digital twins". This demo notebook shows how these tools may address important needs such as:

- flexible, modular design
- interactivity
- scalability and portability (laptop, big workstation, HPC, cloud)
- high-level, user-friendly interface
- data/model integration

In [ ]:
import leafmap
import cmcrameri
import ipyfastscape
import numpy as np
import xarray as xr
import xsimlab as xs
import matplotlib.pyplot as plt

import utils

In [ ]:
from dask.distributed import LocalCluster, Client

In [ ]:
cluster = LocalCluster(threads_per_worker=1)

In [ ]:
client = Client(cluster)
client

## Xarray-simlab: Interactive Modelling



### Build and inspect custom "digital-twins" dynamically

Using [fastscape](https://fastscape.org) + extensions as an example...

In [ ]:
from fastscape.models import basic_model

In [ ]:
basic_model

In [ ]:
basic_model.spl

In [ ]:
basic_model.visualize(show_inputs=True)

In [ ]:
from orographic_precipitation.fastscape_ext import (
    OrographicPrecipitation,
    OrographicDrainageDischarge
)


model = basic_model.update_processes(
    {"rain": OrographicPrecipitation, "drainage": OrographicDrainageDischarge}
)

model

In [ ]:
from paraspec.fastscape_ext import (
    ParapatricSpeciation,
    ParapatricEnvironmentElevation
)


model = model.update_processes(
    {"life": ParapatricSpeciation, "life_env": ParapatricEnvironmentElevation}
)

model

In [ ]:
model.visualize(show_inputs=True)

### Explore models 

In [ ]:
%load_ext xsimlab.ipython

In [ ]:
# %create_setup basic_model -v -d
import xsimlab as xs

ds_in = xs.create_setup(
    model=basic_model,
    clocks={
        'time': np.arange(0, 1e6 + 1e4, 1e4)
    },
    input_vars={
        # nb. of grid nodes in (y, x)
        'grid__shape': [201, 201],
        # total grid length in (y, x)
        'grid__length': [5e4, 5e4],
        # node status at borders
        'boundary__status': 'fixed_value',
        # uplift rate
        'uplift__rate': ('uplift__rate', np.linspace(1e-3, 5e-3, 4)),
        # bedrock channel incision coefficient
        'spl__k_coef': ('spl__k_coef', np.linspace(1e-5, 5e-5, 4)),
        # drainage area exponent
        'spl__area_exp': 0.4,
        # slope exponent
        'spl__slope_exp': 1,
        # diffusivity (transport coefficient)
        'diffusion__diffusivity': 1e-1,
        # random seed
        'init_topography__seed': 1234,
    },
    output_vars={
        'topography__elevation': 'time',
        'erosion__rate': 'time',
        'drainage__area': 'time'
    }
)


In [ ]:
ds_in

In [ ]:
ds_out = (
    ds_in
    .stack(batch=['uplift__rate', 'spl__k_coef'])
    .xsimlab.run(
        model=basic_model, batch_dim='batch', scheduler=client, parallel=True, store="run1.zarr"
    )
    .unstack('batch')
)

In [ ]:
ds_out

In [ ]:
(
    ds_out
     # take topographic elevation
     .topography__elevation
     # take last time step
     .isel(time=-1)
     # extract cross-section
     .sel(x=2.5e4)
     # facetplot for all parameter values
     .plot(row='uplift__rate', col='spl__k_coef')
);

In [ ]:
app = ipyfastscape.TopoViz3d(ds_out, time_dim="time")
app.show()

In [ ]:
app.widget.close()

## Include Data / Observations

Goals: external forcing, data assimilation, comparison, optimization/inference, etc.

Let's use interactive tools!

In [ ]:
m = leafmap.Map(
    google_map="TERRAIN",
    center=[47.04766864046083, 9.4207763671875],
    zoom=7
)

m

### Easy (lazy) access to large (TB) datasets

More and more data (and metadata) available through APIs...

- [SpatioTemporal Asset Catalogs](https://stacspec.org/)
  - Google Earth engine
  - Microsoft Planetary Computer
  - [OpenTopography](https://opentopography.org/)
  - [Swisstopo](https://www.geo.admin.ch/en/home.detail.news.html/geo-internet/news2021/news20210301.html)
  - etc.
- Other APIs

... along with high-level, easy-to-use tools integrated with Xarray (ongoing development effort by a large community)

An example (satellite imagery):

In [ ]:
import satsearch
import stackstac

items = satsearch.Search(
    url="https://earth-search.aws.element84.com/v0",
    intersects=m.draw_features[-1]['geometry'],
    collections=["sentinel-s2-l2a-cogs"],
    datetime="2020-04-01/2020-05-01"
).items()

data = stackstac.stack(items, epsg=32631)

data

In [ ]:
subset = (data
 .sel(band=["B01"])
 .isel(x=slice(0, 3000), y=slice(0, 3000))
 .mean("time")
 .squeeze()
)

subset

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

subset.plot(ax=ax)
ax.set_aspect('equal');

### Example: SRTM data

Not yet using STAC (I haven't found any public API)

In [ ]:
m

In [ ]:
srtm, rect = utils.get_srtm_data(m)

In [ ]:
m.add_layer(rect)

In [ ]:
srtm

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
srtm.plot(ax=ax)
ax.set_aspect('equal');

In [ ]:
in_dem = (
    srtm
    .isel(x=slice(100, -100), y=slice(100, -100))
    .coarsen(x=6, y=6, boundary='trim')
    .mean()
    .astype('d')
)

In [ ]:
in_dem

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
in_dem.plot(ax=ax)
ax.set_aspect('equal');

In [ ]:
from fastscape.processes import SurfaceTopography

@xs.process
class InitDEM:
    """Initialize Fastscape model with a given elevation field."""
    dem = xs.variable(dims=('y', 'x'))
    elevation = xs.foreign(SurfaceTopography, 'elevation', intent='out')
    
    def initialize(self):
        self.elevation = self.dem
        

model = basic_model.update_processes({'init_topography': InitDEM})

In [ ]:
ds_out = (
    ds_in
    .xsimlab.update_vars(
        model=model,
        input_vars={
            'init_topography__dem': in_dem,
            'grid__shape': [in_dem.sizes['y'], in_dem.sizes['x']],
            'grid__length': [in_dem.y.max() - in_dem.y.min(), in_dem.x.max() - in_dem.x.min()],
        }
    )
    .stack(batch=['uplift__rate', 'spl__k_coef'])
    .xsimlab.run(
        model=model, batch_dim="batch", parallel=True, scheduler=client, store='run2.zarr'
    )
    .unstack('batch')
    .assign_coords(x=in_dem.x, y=in_dem.y)
)

In [ ]:
ds_out

In [ ]:
(
    ds_out
     # take topographic elevation
     .topography__elevation
     # take last time step
     .isel(time=-1)
     # facetplot for all parameter values
     .plot(row='uplift__rate', col='spl__k_coef')
);

### Example 2: High-resolution elevation data

Swiss DEM (STAC API)

In [ ]:
m

In [ ]:
items, da = utils.get_swiss_elevation(m)

In [ ]:
m.add_geojson(items.geojson(), layer_name=da.name)

In [ ]:
da

In [ ]:
da.compute().plot(figsize=(8, 7));

### Other data sources

- [RockHound](https://www.fatiando.org/rockhound/latest/index.html): Geophysical datasets

In [ ]:
m

In [ ]:
import rockhound

etopo1 = rockhound.etopo1.fetch_etopo1("bedrock").rio.write_crs("epsg:4326")

roi = m.draw_features[-1]['geometry']['coordinates'][0]
lon, lat = list(zip(*roi))

dem = etopo1.sel(longitude=slice(min(lon), max(lon)), latitude=slice(min(lat), max(lat)))
dem_utm = dem.rio.reproject(dem.rio.estimate_utm_crs())

dem_utm

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))

dem_utm.bedrock.plot(cmap=cmcrameri.cm.bukavu, ax=ax)
ax.set_aspect(1)

- climate data: [NOAA gridded datasets](https://psl.noaa.gov/data/gridded/)

In [ ]:
ds_uv = []

for field in ['uwnd', 'vwnd']:
    base_url = f'http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis/surface/{field}.sig995'
    files = [f'{base_url}.{year}.nc' for year in range(2018, 2019)]
    ds_uv.append(xr.open_mfdataset(files))

ncep_wind = xr.merge(ds_uv)

ncep_wind

In [ ]:
wind_mean = (
    ncep_wind
    .sel(lon=slice(min(lon), max(lon)), lat=slice(max(lat), min(lat)))
    .mean("time")
    .compute()
)

wind_mean

In [ ]:
m

In [ ]:
from ipyleaflet.velocity import Velocity

vel = Velocity(
    data=wind_mean,
    zonal_speed='uwnd',
    meridional_speed='vwnd',
    latitude_dimension='lat',
    longitude_dimension='lon',
    display_options={
        'velocityType': 'Global Wind',
        'displayPosition': 'bottomleft',
        'displayEmptyString': 'No wind data'
    }
)
m.add_layer(vel)

In [ ]:
m.close()

- https://pangaea.de/